In [2]:
# Import needed modules
import pandas as pd

In [3]:
# Import file and look at first 5 rows
filepath = "Resources/purchase_data.csv"
df = pd.read_csv(filepath)
df.head()

Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price  
0  Extraction, Quickblade Of Trembling Hands   3.53  
1                          Frenzied Scimitar   1.56  
2                               Final Critic   4.88  
3                                Blindscythe   3.27  
4                                       Fury   1.44

In [4]:
# Make sure no numeric categories have null values
df.describe()

Purchase ID         Age     Item ID       Price
count   780.000000  780.000000  780.000000  780.000000
mean    389.500000   22.714103   91.755128    3.050987
std     225.310896    6.659444   52.697702    1.169549
min       0.000000    7.000000    0.000000    1.000000
25%     194.750000   20.000000   47.750000    1.980000
50%     389.500000   22.000000   92.000000    3.150000
75%     584.250000   25.000000  138.000000    4.080000
max     779.000000   45.000000  183.000000    4.990000

In [5]:
# Get total Player Count
totalPlayers = pd.DataFrame([{"Total Players" : df["SN"].nunique()}])
totalPlayers


Total Players
0            576

In [6]:
# Purchasing Analysis (Total)
itemIDs = df["Item ID"].nunique()
avgPurchasePrice = df["Price"].mean()
totalPurchases = df["Purchase ID"].count()
totalRev = df["Price"].sum()
# Compile above stats into data frame and format money types to round to 2 decimal places
# Also want to format Total Revenue such that commas appear in the money figure
purchaseAnal = pd.DataFrame({"Number of Unique Items": [itemIDs], "Average Price": ["$"+str(round(avgPurchasePrice,2))],
                            "Number of Purchases" : [totalPurchases], "Total Revenue": ["$"+str("{:,}".format(round(totalRev,2)))]})
purchaseAnal

Number of Unique Items Average Price  Number of Purchases Total Revenue
0                     179         $3.05                  780     $2,379.77

In [7]:
# Gender Demographics
# Cannot just count occurences of "Male" in Gender column, because may have duplicate SNs
# So we create new data frame with only unique SNs 
# Reference: https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/
uniqueSNs = df.sort_values("SN")
uniqueSNs.drop_duplicates(subset = ["SN"],inplace = True)

# get counts of each gender from new data frame
totalentries = uniqueSNs["SN"].count()
countmales = uniqueSNs.Gender.value_counts().Male
countfemales = uniqueSNs.Gender.value_counts().Female
countothers = uniqueSNs["Gender"].count() - countmales - countfemales

# Get percentages
male_p = round(100*countmales / totalentries,2)
female_p = round(100*countfemales / totalentries,2)
other_p = round(100*countothers / totalentries,2)

gender_demo = pd.DataFrame({"Total Count":[countmales, countfemales, countothers],
                           "Percentage of Players":[str(male_p)+"%", str(female_p)+"%", str(other_p)+"%" ]})
# make the gender categories the index
gender_demo.index = ["Male", "Female", "Other / Non-Disclosed"]
gender_demo


Total Count Percentage of Players
Male                           484                84.03%
Female                          81                14.06%
Other / Non-Disclosed           11                 1.91%

In [8]:
# Purchase counts by Gender
pur_counts = df.groupby('Gender')['Purchase ID'].count()

# Average purchase price
avg_pur = df.groupby('Gender')['Price'].mean()
# Format such that we don't lose trailing zero
avg_pur1 = ["$"+"{:.2f}".format((round(i,2))) for i in avg_pur]

# total purchase value
pur_val_avg = df.groupby('Gender')['Price'].sum()
# Format so that numbers > 1000 have comma included
pur_val_avg1 = ["$"+"{:,}".format(round(i,2)) for i in pur_val_avg]

# Average total purchase per person
# Get list of each player, their gender, and their total purchase amount
playertotals = df.groupby(['SN', 'Gender'], as_index=False)['Price'].sum()
# Group resulting data frame by gender and get mean total price
gendertotals = playertotals.groupby('Gender')['Price'].mean()
gendertotals1 = ["$"+"{:,}".format(round(i,2)) for i in gendertotals]


In [9]:
# Make new data frame for Purchasing Analysis by Gender using above info

gender_p = pd.DataFrame({"Purchase Count": pur_counts,
                        "Average Purchase Price": avg_pur1,
                        "Total Purchase Value": pur_val_avg1,
                        "Avg Total Purchase per Person": gendertotals1})

gender_p

Purchase Count Average Purchase Price  \
Gender                                                         
Female                            113                  $3.20   
Male                              652                  $3.02   
Other / Non-Disclosed              15                  $3.35   

                      Total Purchase Value Avg Total Purchase per Person  
Gender                                                                    
Female                             $361.94                         $4.47  
Male                             $1,967.64                         $4.07  
Other / Non-Disclosed               $50.19                         $4.56

In [10]:
# Age Demographics

agebins = [0, 10, 15, 20, 25, 30, 35, 40, 10000 ]
agelabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+" ]

age_df = uniqueSNs.copy()
age_df["Age Group"] = pd.cut(df['Age'], bins=agebins, labels=agelabels, right=False)

# Player counts by age group
age_counts = age_df.groupby('Age Group')['SN'].count()
age_p = [str(round(100*i / totalentries,2))+"%" for i in age_counts]

agedemos = pd.DataFrame({"Total Count": age_counts, "Percentage of Players" : age_p})

agedemos

Total Count Percentage of Players
Age Group                                   
<10                 17                 2.95%
10-14               22                 3.82%
15-19              107                18.58%
20-24              258                44.79%
25-29               77                13.37%
30-34               52                 9.03%
35-39               31                 5.38%
40+                 12                 2.08%

In [11]:
# Purchasing Analytics by Age

df1 = df.copy()
df1["Age Group"] = pd.cut(df['Age'], bins=agebins, labels=agelabels, right=False)
# Purchase counts by age
age1_counts = df1.groupby('Age Group')['Purchase ID'].count()

# Average purchase price
avg_pur2 = df1.groupby('Age Group')['Price'].mean()
# Format such that we don't lose trailing zero
avg_pur3 = ["$"+"{:.2f}".format((round(i,2))) for i in avg_pur2]

# total purchase value
pur_val_avg2 = df1.groupby('Age Group')['Price'].sum()
# Format so that numbers > 1000 have comma included
pur_val_avg3 = ['${:,.2f}'.format(round(i,2)) for i in pur_val_avg2]


# Average total purchase per person
# Get list of each player, their age group, and their total purchase amount

#each individual player total purchase sum
meep = df1.groupby(['SN'])['Price'].sum()
uniqueSNs = uniqueSNs.assign(snSums=meep.values)

# add age bin column to unique SN data frame
uniqueSNs["Age Group"] = pd.cut(uniqueSNs['Age'], bins=agebins, labels=agelabels, right=False)
# group individual total purchase sums by age bins
tots = uniqueSNs.groupby('Age Group')['snSums'].mean()
tots1 = ["$"+"{:.2f}".format((round(i,2))) for i in tots]


In [12]:
# Make new data frame for Purchasing Analysis by Age using above info

age_p = pd.DataFrame({"Purchase Count": age1_counts,
                        "Average Purchase Price": avg_pur3,
                        "Total Purchase Value": pur_val_avg3,
                        "Avg Total Purchase per Person": tots1})

age_p

Purchase Count Average Purchase Price Total Purchase Value  \
Age Group                                                               
<10                    23                  $3.35               $77.13   
10-14                  28                  $2.96               $82.78   
15-19                 136                  $3.04              $412.89   
20-24                 365                  $3.05            $1,114.06   
25-29                 101                  $2.90              $293.00   
30-34                  73                  $2.93              $214.00   
35-39                  41                  $3.60              $147.67   
40+                    13                  $2.94               $38.24   

          Avg Total Purchase per Person  
Age Group                                
<10                               $4.54  
10-14                             $3.76  
15-19                             $3.86  
20-24                             $4.32  
25-29                             $3.81  
30-34                             $4.12  
35-39                             $4.76  
40+                               $3.19

In [13]:
# Top Spenders
# need total purchase count for inidividual players
meep1 = df1.groupby(['SN'])['Purchase ID'].count()
uniqueSNs = uniqueSNs.assign(snPurchaseCounts=meep1.values)

# need average purchase price grouped by individual player
meep2 = df1.groupby(['SN'])['Price'].mean()
meep3 = ["$"+"{:.2f}".format((round(i,2))) for i in meep2]
uniqueSNs = uniqueSNs.assign(snAvgPrice=meep3)

# Get stats for top 5 spenders
topSpenders = uniqueSNs.copy()

topSpenders1 = topSpenders.nlargest(5, 'snSums')
topSpenders1['Total Purchase Value'] = topSpenders1['snSums'].map('${:,.2f}'.format)
topSpenders1 = topSpenders1.drop(columns=['Purchase ID', 'Age', 'Gender', 'Item ID',
                                         'Item Name', 'Age Group', 'Price', 'snSums'])
topSpenders1 = topSpenders1.set_index('SN')
topSpenders1 = topSpenders1.rename(columns={"snPurchaseCounts": "Purchase Count", "snAvgPrice": "Average Purchase Price"})
topSpenders1

Purchase Count Average Purchase Price Total Purchase Value
SN                                                                     
Lisosia93                 5                  $3.79               $18.96
Idastidru52               4                  $3.86               $15.45
Chamjask73                3                  $4.61               $13.83
Iral74                    4                  $3.40               $13.62
Iskadarya95               3                  $4.37               $13.10

In [15]:
# Most popular items
mostpop = df.copy()

# Get counts of each item ID and pull 5 largest
m1 = pd.DataFrame(mostpop.groupby(['Item ID'])['Item Name'].value_counts())
m1 = m1.rename(columns={"Item Name": "Purchase Count"})
m1 = m1.sort_values(by=['Purchase Count'], ascending=False)
m1 = m1.head()

# merge the counts data frame 'm1' with the mostpop data frame so we can get aggregate values
merged = pd.merge(m1, mostpop, on='Item ID', how='left')
merged = merged.set_index(['Item ID', 'Item Name'])
avgprices = pd.DataFrame(merged.groupby(['Item ID', 'Item Name'])['Price'].mean())
sumprices = pd.DataFrame(merged.groupby(['Item ID', 'Item Name'])['Price'].sum())

# Add avg and total purchase price columns to purchase count table and format
m1['Item Price'] = avgprices.head()
m1['Total Purchase Value'] = sumprices.head()
m1['Item Price'] = m1['Item Price'].map('${:,.2f}'.format)
m1['Total Purchase Value'] = m1['Total Purchase Value'].map('${:,.2f}'.format)
m1

Purchase Count  \
Item ID Item Name                                                      
92      Final Critic                                              13   
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
132     Persuasion                                                 9   
108     Extraction, Quickblade Of Trembling Hands                  9   

                                                     Item Price  \
Item ID Item Name                                                 
92      Final Critic                                      $4.61   
178     Oathbreaker, Last Hope of the Breaking Storm      $4.23   
145     Fiery Glass Crusader                              $4.58   
132     Persuasion                                        $3.22   
108     Extraction, Quickblade Of Trembling Hands         $3.53   

                                                     Total Purchase Value  
Item ID Item Name                                                          
92      Final Critic                                               $59.99  
178     Oathbreaker, Last Hope of the Breaking Storm               $50.76  
145     Fiery Glass Crusader                                       $41.22  
132     Persuasion                                                 $28.99  
108     Extraction, Quickblade Of Trembling Hands                  $31.77

In [22]:
mostpop

Purchase ID             SN  Age  Gender  Item ID  \
0              0        Lisim78   20    Male      108   
1              1    Lisovynya38   40    Male      143   
2              2     Ithergue48   24    Male       92   
3              3  Chamassasya86   24    Male      100   
4              4      Iskosia90   23    Male      131   
..           ...            ...  ...     ...      ...   
775          775     Aethedru70   21  Female       60   
776          776         Iral74   21    Male      164   
777          777     Yathecal72   20    Male       67   
778          778        Sisur91    7    Male       92   
779          779      Ennrian78   24    Male       50   

                                     Item Name  Price  
0    Extraction, Quickblade Of Trembling Hands   3.53  
1                            Frenzied Scimitar   1.56  
2                                 Final Critic   4.88  
3                                  Blindscythe   3.27  
4                                         Fury   1.44  
..                                         ...    ...  
775                                       Wolf   3.54  
776                           Exiled Doomblade   1.63  
777      Celeste, Incarnation of the Corrupted   3.46  
778                               Final Critic   4.19  
779                                       Dawn   4.60  

[780 rows x 7 columns]

In [28]:
# Most Profitable Items

# Can still use mostpop data frame for this
m2 = pd.DataFrame(mostpop.groupby(['Item ID'])['Price'].sum())
m2 = m2.rename(columns={"Price": "Item Price"})
m2 = m2.sort_values(by=['Item Price'], ascending=False)
m2 = m2.head()

# merge the counts data frame 'm2' with the mostpop data frame so we can get aggregate values
merged2 = pd.merge(m2, mostpop, on='Item ID', how='left')
merged2 = merged2.set_index(['Item ID', 'Item Price'])
merged2


Purchase ID              SN  Age  Gender  \
Item ID Item Price                                             
92      59.99                 2      Ithergue48   24    Male   
        59.99                99    Haisrisuir60   23    Male   
        59.99               252        Tyaelo67   39    Male   
        59.99               273        Phyali88   15  Female   
        59.99               277      Ennalmol65   24    Male   
        59.99               411        Lisico81   10    Male   
        59.99               536     Siallylis44   20    Male   
        59.99               712       Lisilsa62   25    Male   
        59.99               722        Ilarin91   22    Male   
        59.99               767         Ilmol66    8  Female   
        59.99               768    Assassasta79   38    Male   
        59.99               774      Jiskjask80   11    Male   
        59.99               778         Sisur91    7    Male   
178     50.76                25       Lisirra87   29    Male   
        50.76                61      Jiskimya77   17    Male   
        50.76                62      Yadaphos40   30    Male   
        50.76                72       Marilsa69   25  Female   
        50.76               222      Chamjask73   22  Female   
        50.76               464    Rianistast50   22    Male   
        50.76               580         Tyida79   24    Male   
        50.76               627          Arin32   25    Male   
        50.76               658      Quilassa66    7  Female   
        50.76               706   Chanossanya44   20    Male   
        50.76               730       Aidaira26   30    Male   
        50.76               753  Frichirranya75   36    Male   
82      44.10                18       Reunasu60   22  Female   
        44.10                88      Laedallo55   30    Male   
        44.10               202       Phistym51   37    Male   
        44.10               240    Lirtastsda29   16    Male   
        44.10               247     Iskadarya95   20    Male   
        44.10               274   Aerithllora36   29  Female   
        44.10               504    Chamilsala65   23  Female   
        44.10               522     Meosridil82   18    Male   
        44.10               666      Assilsan72   20  Female   
145     41.22               132       Inguron55   23    Male   
        41.22               207    Sondastsda82   24    Male   
        41.22               329        Ceoral34   20    Male   
        41.22               438       Qilunan34   23    Male   
        41.22               474           Ina92   31    Male   
        41.22               572   Yoishirrala98   17  Female   
        41.22               683       Ialallo29   15  Female   
        41.22               741    Phiallylis33   19    Male   
        41.22               769       Ilosian36   15    Male   
103     34.80               175        Indcil77   17    Male   
        34.80               373          Aina42   25    Male   
        34.80               455     Lisossala30   20    Male   
        34.80               460    Assistasda90   25    Male   
        34.80               476       Marassa62   21    Male   
        34.80               634         Eoral49   16    Male   
        34.80               660          Isri34   38    Male   
        34.80               714       Tyidaim51   16  Female   

                                                       Item Name  Price  
Item ID Item Price                                                       
92      59.99                                       Final Critic   4.88  
        59.99                                       Final Critic   4.19  
        59.99                                       Final Critic   4.88  
        59.99                                       Final Critic   4.88  
        59.99                                       Final Critic   4.88  
        59.99                                       Final Critic   4.19  
        59.99                      